In [1]:
import viz_utils
from equilibrator_api import ComponentContribution, Q_
import cobra
import numpy as np
import pickle
from math import exp
import requests
import importlib

In [2]:
importlib.reload(viz_utils)
from viz_utils import print_reactions

In [3]:
def build_dict_compounds(list_compounds, dict_compounds = {}):
    """
    """
    for compound in list_compounds:
        url = "http://rest.kegg.jp/get/"
        r = requests.get(url=url+compound)
        car = str(r.content)[1]
        info = str(r.content).split(car)[1].split('\\n')
        dict_compounds[compound] = info[1].split(None, 1)[1].split(';')[0]

    return dict_compounds

In [4]:
cc = ComponentContribution()

In [5]:
model = cobra.io.read_sbml_model("../data/Models/ccm_ross_updated.xml")

In [6]:
metabolites = [met for met in model.metabolites]

In [7]:
len(metabolites)

60

In [8]:
reactions = [react for react in model.reactions]
len(reactions)

69

In [9]:
with open("../data/compound.p", 'rb') as c:
    dict_compound = pickle.load(c)

compounds = list(dict_compound.keys())

In [10]:
dc = build_dict_compounds(compounds)
dc

{'C00001': 'H2O',
 'C00002': 'ATP',
 'C00003': 'NAD+',
 'C00004': 'NADH',
 'C00005': 'NADPH',
 'C00006': 'NADP+',
 'C00007': 'Oxygen',
 'C00008': 'ADP',
 'C00009': 'Orthophosphate',
 'C00010': 'CoA',
 'C00011': 'CO2',
 'C00022': 'Pyruvate',
 'C00024': 'Acetyl-CoA',
 'C00026': '2-Oxoglutarate',
 'C00036': 'Oxaloacetate',
 'C00042': 'Succinate',
 'C00058': 'Formate',
 'C00074': 'Phosphoenolpyruvate',
 'C00080': 'H+',
 'C00084': 'Acetaldehyde',
 'C00085': 'D-Fructose 6-phosphate',
 'C00091': 'Succinyl-CoA',
 'C00111': 'Glycerone phosphate',
 'C00117': 'D-Ribose 5-phosphate',
 'C00118': 'D-Glyceraldehyde 3-phosphate',
 'C00122': 'Fumarate',
 'C00149': '(S)-Malate',
 'C00158': 'Citrate',
 'C00197': '3-Phospho-D-glycerate',
 'C00199': 'D-Ribulose 5-phosphate',
 'C00227': 'Acetyl phosphate',
 'C00231': 'D-Xylulose 5-phosphate',
 'C00236': '3-Phospho-D-glyceroyl phosphate',
 'C00311': 'Isocitrate',
 'C00345': '6-Phospho-D-gluconate',
 'C00354': 'D-Fructose 1,6-bisphosphate',
 'C00390': 'Ubiqui

In [12]:
len(dc)

46

In [11]:
dict_compound = {name:id for id, name in dc.items()}
dict_compound

{'H2O': 'C00001',
 'ATP': 'C00002',
 'NAD+': 'C00003',
 'NADH': 'C00004',
 'NADPH': 'C00005',
 'NADP+': 'C00006',
 'Oxygen': 'C00007',
 'ADP': 'C00008',
 'Orthophosphate': 'C00009',
 'CoA': 'C00010',
 'CO2': 'C00011',
 'Pyruvate': 'C00022',
 'Acetyl-CoA': 'C00024',
 '2-Oxoglutarate': 'C00026',
 'Oxaloacetate': 'C00036',
 'Succinate': 'C00042',
 'Formate': 'C00058',
 'Phosphoenolpyruvate': 'C00074',
 'H+': 'C00080',
 'Acetaldehyde': 'C00084',
 'D-Fructose 6-phosphate': 'C00085',
 'Succinyl-CoA': 'C00091',
 'Glycerone phosphate': 'C00111',
 'D-Ribose 5-phosphate': 'C00117',
 'D-Glyceraldehyde 3-phosphate': 'C00118',
 'Fumarate': 'C00122',
 '(S)-Malate': 'C00149',
 'Citrate': 'C00158',
 '3-Phospho-D-glycerate': 'C00197',
 'D-Ribulose 5-phosphate': 'C00199',
 'Acetyl phosphate': 'C00227',
 'D-Xylulose 5-phosphate': 'C00231',
 '3-Phospho-D-glyceroyl phosphate': 'C00236',
 'Isocitrate': 'C00311',
 '6-Phospho-D-gluconate': 'C00345',
 'D-Fructose 1,6-bisphosphate': 'C00354',
 'Ubiquinol': 'C00

In [12]:
dict_equations = {'name': [], 'list': [], 'id': []}

for r in model.reactions:
    a, b, c = print_reactions(r,0,False)

    eq = ' '.join(c)
    dict_equations['name'].append(eq)
    dict_equations['list'].append(c)

    # metabolites = [m for m in r.metabolites]
    # coefficients = [abs(c) for c in r.get_coefficients(metabolites)]
    # for coeff in coefficients:
    #     a = "".join(a.split(str(coeff)+" "))
    # print(a)
    # equations.append(a)   


In [13]:
# eq_request = []
for eq in dict_equations['list']:
    new_eq = []
    # print(eq)
    
    for compound in eq:
        done = False
        if compound in dict_compound:
            # print(f'{c}:{dict_compound[c]}')
            new_eq.append(f"kegg:{dict_compound[compound]}")
        elif compound in ['+', '<=>']:
            new_eq.append(compound)
        else:
            dict_equations['id'].append("")
            # new_eq.append(compound)
            break
        done = True
    if done and len(new_eq) > 1:
        dict_equations['id'].append(' '.join(new_eq))
    

In [14]:
for index in range(len(dict_equations['name'])):
    try:
        reaction = cc.search_reaction(dict_equations['name'][index])
        # print(f"Results from equilibrator for {dict_equations['name'][index]}")
        # dg = cc.standard_dg_prime(reaction)
        # print(exp(-((dg.value)/cc.RT)))
    except:
        # print(f"Trying {dict_equations['id'][index]}")
        try:
            reaction = cc.search_reaction(dict_equations['id'][index])
            # print(reaction)
            # dg = cc.standard_dg_prime(reaction)
            # print(exp(-((dg.value)/cc.RT)))
            # print(f"Results from equilibrator for {dict_equations['name'][index]}")
        except:
            print(f"No results from equilibrator for {dict_equations['name'][index]}")
            print(f"No results from equilibrator for {dict_equations['id'][index]}")
    # dg = cc.sta

No results from equilibrator for ACETATE-ext <=>
No results from equilibrator for 
No results from equilibrator for CO2-ext <=>
No results from equilibrator for 
No results from equilibrator for ETOH-ext <=>
No results from equilibrator for 
No results from equilibrator for FORMATE-ext <=>
No results from equilibrator for 
No results from equilibrator for GLU-ext <=>
No results from equilibrator for 
No results from equilibrator for LACTATE-ext <=>
No results from equilibrator for 
No results from equilibrator for OXY-ext <=>
No results from equilibrator for 
No results from equilibrator for H-ext <=>
No results from equilibrator for 
No results from equilibrator for SUCC-ext <=>
No results from equilibrator for 
No results from equilibrator for H2O <=>
No results from equilibrator for kegg:C00001 <=>
No results from equilibrator for Orthophosphate <=>
No results from equilibrator for kegg:C00009 <=>
No results from equilibrator for BIOMASS <=>
No results from equilibrator for 


In [202]:
r = cc.parse_reaction_formula('Acetyl phosphate + ADP <=> Acetate + ATP')

ParseException: Non-specific reaction: Acetyl phosphate + ADP

In [64]:
dg = cc.standard_dg_prime(r)

In [17]:
'Citrate <=>  Isocitrate'.split(None, 1)

['Citrate', '<=>  Isocitrate']

In [79]:
exp(-((dg.value)/cc.RT))

0.00023739205221358383

In [22]:
import re

pattern = re.compile('(?P<reac>\w*) *(?P<op>\w*) *(?P<prod>\w*)')


In [23]:

re.findall(pattern, 'Citrate <=>  Isocitrate')


[('', '', 'Isocitrate')]